In [130]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import random as rand

import keras.backend as K
import tensorflow as tf

from keras.models import Model
from keras.layers import Input, Dense, Lambda

# Brown dataset: vectorization using doc2vec

In [111]:
with open('data/discourse_markers/X_50dim_d2v.pkl', 'rb') as f:
    X = pickle.load(f)
with open('data/discourse_markers/y.pkl', 'rb') as f:
    y = pickle.load(f)

In [125]:
# number of non-false items
num_true = 4000 #sum([sum(x) != 0 for x in y])

balanced_X = np.zeros([num_true * 2, X.shape[1]])
balanced_y = np.zeros([num_true * 2, y.shape[1]])

tr_idx = 0
fa_idx = 0
overall_idx = 0

for idx in range(len(X)):
    if sum(y[idx]) > 1:
        continue
    if tr_idx < num_true and sum(y[idx]) != 0:
        balanced_X[overall_idx, :] = X[idx]
        balanced_y[overall_idx, :] = y[idx]
        tr_idx += 1
        overall_idx += 1
    if fa_idx < num_true and sum(y[idx]) == 0:
        balanced_X[overall_idx, :] = X[idx]
        balanced_y[overall_idx, :] = y[idx]
        fa_idx += 1
        overall_idx += 1

In [126]:
X_train, X_test, y_train, y_test = train_test_split(balanced_X, balanced_y, test_size=0.1, random_state=42)

In [116]:
batch_size = 32
num_units = 128
vector_size = X.shape[1] # 50
num_outputs = y.shape[1] # 59

In [139]:
K.clear_session()

In [140]:
input_layer = Input(shape = (vector_size,), dtype = 'float32', name = 'input_layer')
dense_1 = Dense(num_units, activation = 'relu', name = 'dense_1')(input_layer)
dense_2 = Dense(num_units, activation = 'relu', name = 'dense_2')(dense_1)
output = Dense(num_outputs, activation = 'softmax', name = 'output_layer')(dense_2)

#output = Dense(num_outputs, activation = 'sigmoid', name = 'output_layer')(dense_2)

#out_and = Lambda(lambda x: x[..., 0:1], name = 'and')(output)
#out_so = Lambda(lambda x: x[..., 1:2], name = 'so')(output)

#model = Model(inputs = input_layer, outputs = [out_and, out_so])
#model.compile(optimizer = 'adam',
#              loss = ['binary_crossentropy', 'binary_crossentropy'],
#              metrics = ['accuracy'])

model = Model(inputs = input_layer, outputs = output)

model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [131]:
c_weights = []
c_weights.append(class_weight.compute_class_weight('balanced_and'
                                               ,np.unique(y_train[..., 0:1])
                                               ,y_train[..., 0:1]))
c_weights.append(class_weight.compute_class_weight('balanced_and'
                                               ,np.unique(y_train[..., 1:2])
                                               ,y_train[..., 1:2]))

TypeError: unhashable type: 'numpy.ndarray'

In [141]:
history = model.fit(X_train, y_train, 
                    validation_split = 0.3, epochs = 20, batch_size = 32, verbose = 2)

Train on 5040 samples, validate on 2160 samples
Epoch 1/20
 - 1s - loss: 1.4258 - acc: 0.2286 - val_loss: 1.2097 - val_acc: 0.5593
Epoch 2/20
 - 1s - loss: 1.2472 - acc: 0.2595 - val_loss: 1.1816 - val_acc: 0.3009
Epoch 3/20
 - 1s - loss: 1.2262 - acc: 0.3036 - val_loss: 1.1789 - val_acc: 0.1699
Epoch 4/20
 - 1s - loss: 1.2056 - acc: 0.2750 - val_loss: 1.1677 - val_acc: 0.3671
Epoch 5/20
 - 1s - loss: 1.1831 - acc: 0.3014 - val_loss: 1.1562 - val_acc: 0.3639
Epoch 6/20
 - 1s - loss: 1.1614 - acc: 0.3131 - val_loss: 1.1540 - val_acc: 0.3620
Epoch 7/20
 - 1s - loss: 1.1372 - acc: 0.3101 - val_loss: 1.1588 - val_acc: 0.3046
Epoch 8/20
 - 1s - loss: 1.1129 - acc: 0.3185 - val_loss: 1.1755 - val_acc: 0.5319
Epoch 9/20
 - 1s - loss: 1.0965 - acc: 0.3288 - val_loss: 1.1624 - val_acc: 0.2412
Epoch 10/20
 - 1s - loss: 1.0711 - acc: 0.3145 - val_loss: 1.1672 - val_acc: 0.3986
Epoch 11/20
 - 1s - loss: 1.0499 - acc: 0.3387 - val_loss: 1.1757 - val_acc: 0.2412
Epoch 12/20
 - 1s - loss: 1.0319 - ac

In [142]:
model.evaluate(X_test, y_test, verbose=2)

[1.346786060333252, 0.29]

In [86]:
for x in brown_disc_df:
    print(x)

sent
And
So
But
Plus
Now
Or
After all
Sure
Of course
By the way
Yet
Okay
Ok
Well
Again
Since then
Basically
As such
Oh
Not only that
As a result
Anywho
Anyhow
Anyway
Ah
So there you have it
Finally
For starters
At least
Seriously
In fact
I'd argue
You know
You see
I mean
If so
Beyond that
Alas
That way
All told
Yes
Hey
That's ok
That's okay
That said
With that said
Uh
Admittedly
Unfortunately
Fortunately
First
Recently
Here goes...
Heck
In short
First of all
That is
For example
Also
clean
